# Lấy dữ liệu từ Materials Project API

In [3]:
import os
from mp_api.client import MPRester
import pandas as pd

# --- CẤU HÌNH ---
# API Key của tôi
MY_API_KEY = os.getenv("MP_API_KEY")

# --- HÀM KÉO DỮ LIỆU ---
def fetch_materials_data(api_key):
    print("Đang kết nối tới Materials Project...")
    
    with MPRester(api_key) as mpr:
        # Tìm kiếm vật liệu: Band gap = 0 (Kim loại)
        
        docs = mpr.summary.search(
            band_gap=(0, 0.0), 
            fields=[
                "material_id", 
                "formula_pretty", 
                "composition",
                "density", 
                "formation_energy_per_atom",
                "band_gap", 
                "volume",
                "nelements", 
                "structure" 
            ]
        )
        
    print(f"Hốt được {len(docs)} vật liệu. Bắt đầu xử lý dữ liệu...")
    return docs

# --- XỬ LÝ DỮ LIỆU RA CSV ---
def save_to_csv(docs, filename="../data/data_Materials Project/raw_materials_data.csv"):
    if not docs:
        print("Không có dữ liệu để lưu.")
        return

    data_list = []
    
    print("Đang xử lý dữ liệu...")
    for doc in docs:
        try:
            # Trích xuất dữ liệu
            item = {
                "id": doc.material_id,
                "formula": doc.formula_pretty,
                "density": doc.density,
                "formation_energy": doc.formation_energy_per_atom,
                "band_gap": doc.band_gap,
                "volume": doc.volume,
                "num_elements": doc.nelements,
                # Lấy thông tin cấu trúc mạng tinh thể (Lattice)
                "lattice_a": doc.structure.lattice.a,
                "lattice_b": doc.structure.lattice.b,
                "lattice_c": doc.structure.lattice.c,
                "lattice_alpha": doc.structure.lattice.alpha,
                "lattice_beta": doc.structure.lattice.beta,
                "lattice_gamma": doc.structure.lattice.gamma,
            }
            data_list.append(item)
        except Exception as e:
            print(f"Lỗi khi xử lý vật liệu {doc.material_id}: {e}")
            continue
    
    # Tạo DataFrame và xuất file
    df = pd.DataFrame(data_list)
    df.to_csv(filename, index=False)
    print(f"Xong! Dữ liệu đã nằm trong file '{filename}'.")
    

# --- CHẠY --- 
if __name__ == "__main__":
    try:
        # Bước 1: Kéo data bằng cái hàm đã viết đàng hoàng ở trên
        materials = fetch_materials_data(MY_API_KEY)
        
        # Bước 2: Lưu file
        save_to_csv(materials)
            
    except Exception as e:
        print(f"Lỗi rồi: {e}")
        print("Check lại API Key hoặc mạng đi nhé!")

Đang kết nối tới Materials Project...


C:\Users\pikal\AppData\Local\Temp\ipykernel_8556\759389017.py:16: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  docs = mpr.summary.search(


Retrieving SummaryDoc documents:   0%|          | 0/72640 [00:00<?, ?it/s]

Hốt được 72640 vật liệu. Bắt đầu xử lý dữ liệu...
Đang xử lý dữ liệu...
Xong! Dữ liệu đã nằm trong file '../data/data_Materials Project/raw_materials_data.csv'.
